In [116]:
from datetime import datetime, timedelta
import yfinance as yf

In [117]:
def get_stock_data(stock_ticker, date):
    endDate = datetime.now()
    stock = yf.Ticker(stock_ticker)
    hist = stock.history(start=date, end=endDate)
   
    if not hist.empty:
        closest_date = hist.index[0]
        open_price = hist.iloc[0]["Open"]
        High_price = hist.iloc[0]["High"]
        Low_price = hist.iloc[0]["Low"]
        close_price = hist.iloc[0]["Close"]
        volume = hist.iloc[0]["Volume"]
        price_movement_per = ((close_price - open_price) / open_price) * 100
        stock_movement = "up" if close_price > open_price else "down"
        return {
            "stock_ticker": stock_ticker,
            "date": closest_date.strftime("%Y-%m-%d"),
            "open_price": open_price,
            "High_price": High_price,
            "Low_price": Low_price,
            "close_price": close_price,
            "volume": volume,
            "movement": stock_movement,
            "price_movement_per": price_movement_per,
        }
    else:
        return {"error":f"no data available"}

In [118]:
data = get_stock_data("COALINDIA.NS", "2025-01-06")

In [119]:
data

{'stock_ticker': 'COALINDIA.NS',
 'date': '2025-01-06',
 'open_price': 394.04998779296875,
 'High_price': 394.04998779296875,
 'Low_price': 377.1499938964844,
 'close_price': 378.75,
 'volume': 8773112.0,
 'movement': 'down',
 'price_movement': -3.8827530178753014}

In [123]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

def get_stock_features(ticker, news_date):
    # Define date ranges
    news_date = datetime.strptime(news_date, "%Y-%m-%d")
    before_start = (news_date - timedelta(days=5)).strftime("%Y-%m-%d")
    after_end = (news_date + timedelta(days=5)).strftime("%Y-%m-%d")
    
    # Fetch historical data
    stock = yf.Ticker(ticker)
    historical_data = stock.history(start=before_start, end=after_end)
    
    # Convert index to timezone-naive to match news_date
    historical_data.index = historical_data.index.tz_localize(None)
    
    # Ensure data is available
    if historical_data.empty:
        return None
    
    # Compute before-news metrics (5 days)
    before_news = historical_data.loc[:news_date - timedelta(days=1)]
    avg_price_5d = before_news['Close'].mean()
    vol_5d = before_news['Close'].std()
    avg_volume_5d = before_news['Volume'].mean()
    
    # On news day
    if news_date.strftime("%Y-%m-%d") in historical_data.index:
        news_day = historical_data.loc[news_date.strftime("%Y-%m-%d")]
        open_price = news_day['Open']
        close_price = news_day['Close']
        volume_news_day = news_day['Volume']
    else:
        open_price = close_price = volume_news_day = None
    
    # Compute after-news metrics (1, 3, 5 days)
    after_news = historical_data.loc[news_date + timedelta(days=1):]
    price_movement_1d = ((after_news['Close'].iloc[0] - open_price) / open_price) * 100 if len(after_news) > 0 and open_price else None
    price_movement_3d = ((after_news['Close'].iloc[2] - open_price) / open_price) * 100 if len(after_news) > 2 and open_price else None
    price_movement_5d = ((after_news['Close'].iloc[4] - open_price) / open_price) * 100 if len(after_news) > 4 and open_price else None
    avg_volume_after = after_news['Volume'].mean() if len(after_news) > 0 else None
    
    return {
        "avg_price_5d": avg_price_5d,
        "volatility_5d": vol_5d,
        "avg_volume_5d": avg_volume_5d,
        "open_price_news_day": open_price,
        "close_price_news_day": close_price,
        "volume_news_day": volume_news_day,
        "price_movement_1d": price_movement_1d,
        "price_movement_3d": price_movement_3d,
        "price_movement_5d": price_movement_5d,
        "avg_volume_after": avg_volume_after,
    }


In [124]:
news_date = "2025-01-03"
ticker = "COALINDIA.NS"
features = get_stock_features(ticker, news_date)
print(features)


{'avg_price_5d': 385.3499984741211, 'volatility_5d': 6.213031421149259, 'avg_volume_5d': 10284797.75, 'open_price_news_day': 393.0, 'close_price_news_day': 393.6499938964844, 'volume_news_day': 7122319.0, 'price_movement_1d': -3.6259541984732824, 'price_movement_3d': None, 'price_movement_5d': None, 'avg_volume_after': 8773112.0}


In [125]:
features

{'avg_price_5d': 385.3499984741211,
 'volatility_5d': 6.213031421149259,
 'avg_volume_5d': 10284797.75,
 'open_price_news_day': 393.0,
 'close_price_news_day': 393.6499938964844,
 'volume_news_day': 7122319.0,
 'price_movement_1d': -3.6259541984732824,
 'price_movement_3d': None,
 'price_movement_5d': None,
 'avg_volume_after': 8773112.0}

In [120]:
# import yfinance as yf

# # CREATE A TICKER INSTANCE PASSING TESLA AS THE TARGET COMPANY
# tsla = yf.Ticker('TSLA')

# # CALL THE MULTIPLE FUNCTIONS AVAILABLE AND STORE THEM IN VARIABLES.
# actions = tsla.get_actions()
# analysis = tsla.get_analysis()
# balance = tsla.get_balance_sheet()
# calendar = tsla.get_calendar()
# cf = tsla.get_cashflow()
# info = tsla.get_info()
# inst_holders = tsla.get_institutional_holders()
# news = tsla.get_news()
# recommendations = tsla.get_recommendations()
# sustainability = tsla.get_sustainability()

# # PRINT THE RESULTS
# print(actions)
# print('*'*20)
# print(analysis)
# print('*'*20)
# print(balance)
# print('*'*20)
# print(calendar)
# print('*'*20)
# print(cf)
# print('*'*20)
# print(info)
# print('*'*20)
# print(inst_holders)
# print('*'*20)
# print(news)
# print('*'*20)
# print(recommendations)
# print('*'*20)
# print(sustainability)
# print('*'*20)